In [1]:
import pandas as pd
import requests
import json

In [2]:
lowerToUpperDf = pd.read_csv("/data/Lower_Tier_Local_Authority_to_Upper_Tier_Local_Authority__December_2017__Lookup_in_England_and_Wales.csv")

FileNotFoundError: [Errno 2] File Lower_Tier_Local_Authority_to_Upper_Tier_Local_Authority__December_2017__Lookup_in_England_and_Wales.csv does not exist: 'Lower_Tier_Local_Authority_to_Upper_Tier_Local_Authority__December_2017__Lookup_in_England_and_Wales.csv'

In [3]:
request = requests.get("https://coronavirus.data.gov.uk/downloads/json/coronavirus-cases_latest.json")
requestJson = json.loads(request.content)
ltlasDf = pd.DataFrame(requestJson['ltlas'])
countriesDf = pd.DataFrame(requestJson['countries']) 
regionsDf = pd.DataFrame(requestJson['regions'])
utlasDf = pd.DataFrame(requestJson['utlas'])

In [4]:
print(ltlasDf.shape, countriesDf.shape, regionsDf.shape, utlasDf.shape)

(20275, 10) (106, 10) (805, 10) (10832, 10)


In [5]:
ltlasDf.specimenDate = pd.to_datetime(ltlasDf.specimenDate)
countriesDf.specimenDate = pd.to_datetime(countriesDf.specimenDate)
regionsDf.specimenDate = pd.to_datetime(regionsDf.specimenDate)
utlasDf.specimenDate = pd.to_datetime(utlasDf.specimenDate)

In [6]:
countriesDf.head(5)

,areaCode,areaName,specimenDate,dailyLabConfirmedCases,previouslyReportedDailyCases,changeInDailyCases,totalLabConfirmedCases,previouslyReportedTotalCases,changeInTotalCases,dailyTotalLabConfirmedCasesRate
0,E92000001,England,2020-05-25,26,0,26,150294,149802,492,268.5
1,E92000001,England,2020-05-24,149,18,131,150268,149802,466,268.4
2,E92000001,England,2020-05-23,309,152,157,150119,149784,335,268.2
3,E92000001,England,2020-05-22,609,489,120,149810,149632,178,267.6
4,E92000001,England,2020-05-21,671,628,43,149201,149143,58,266.5


In [7]:
ltlasDf = ltlasDf[['areaCode', 'areaName', 'specimenDate', 'dailyLabConfirmedCases', 'totalLabConfirmedCases']]

In [8]:
ltlasDf.head()

,areaCode,areaName,specimenDate,dailyLabConfirmedCases,totalLabConfirmedCases
0,E09000033,Westminster,2020-05-25,0.0,654
1,E09000032,Wandsworth,2020-05-25,0.0,995
2,E09000031,Waltham Forest,2020-05-25,0.0,763
3,E09000030,Tower Hamlets,2020-05-25,0.0,632
4,E09000029,Sutton,2020-05-25,0.0,735


In [9]:
lowerToUpperDf.head()

NameError: name 'lowerToUpperDf' is not defined

In [10]:
ltlasDf = pd.merge(left=ltlasDf, 
        right=lowerToUpperDf, 
        left_on="areaCode",
        right_on="LTLA17CD")
ltlasDf = ltlasDf.drop(['LTLA17CD','LTLA17NM','FID'], axis=1)

NameError: name 'lowerToUpperDf' is not defined

In [11]:
ltlasDf = ltlasDf.rename(columns={"UTLA17CD" : "upperRegionCode", 'UTLA17NM' : "upperRegionName"})

In [12]:
ltlasDf[ltlasDf.areaName == "Warwick"].head(10)

,areaCode,areaName,specimenDate,dailyLabConfirmedCases,totalLabConfirmedCases
89,E07000222,Warwick,2020-05-25,0.0,295
346,E07000222,Warwick,2020-05-24,1.0,295
781,E07000222,Warwick,2020-05-21,5.0,294
980,E07000222,Warwick,2020-05-20,1.0,289
1200,E07000222,Warwick,2020-05-19,2.0,288
1445,E07000222,Warwick,2020-05-18,4.0,286
1669,E07000222,Warwick,2020-05-17,1.0,282
2078,E07000222,Warwick,2020-05-15,2.0,281
2302,E07000222,Warwick,2020-05-14,2.0,279
2544,E07000222,Warwick,2020-05-13,1.0,277


In [13]:
ltlasDf[ltlasDf.upperRegionName == "Warwickshire"]

AttributeError: 'DataFrame' object has no attribute 'upperRegionName'

In [14]:
utlasDf = utlasDf[['areaCode', 'areaName', 'specimenDate', 'dailyLabConfirmedCases', 'totalLabConfirmedCases']]


In [15]:
ltlasDf = pd.merge(left=ltlasDf,
        right=utlasDf,
        left_on=['upperRegionCode','specimenDate'],
        right_on=['areaCode','specimenDate'],
        suffixes=("","_upperRegion"))

KeyError: 'upperRegionCode'

In [16]:
ltlasDf = ltlasDf.drop(['areaCode_upperRegion', 'areaName_upperRegion'], axis=1) 

KeyError: "['areaCode_upperRegion' 'areaName_upperRegion'] not found in axis"

In [17]:
ltlasDf[ltlasDf.areaName=="Warwick"]

,areaCode,areaName,specimenDate,dailyLabConfirmedCases,totalLabConfirmedCases
89,E07000222,Warwick,2020-05-25,0.0,295
346,E07000222,Warwick,2020-05-24,1.0,295
781,E07000222,Warwick,2020-05-21,5.0,294
980,E07000222,Warwick,2020-05-20,1.0,289
1200,E07000222,Warwick,2020-05-19,2.0,288
...,...,...,...,...,...
17571,E07000222,Warwick,2020-03-22,2.0,11
18061,E07000222,Warwick,2020-03-20,2.0,9
18295,E07000222,Warwick,2020-03-19,2.0,7
18523,E07000222,Warwick,2020-03-18,3.0,5
